In [2]:
import pandas as pd
import feather
from extras import *
pd.set_option('display.max_columns', None)

In [3]:
# Players using a new defined class for their data
ja = Player('Ja Morant', '1629630', '1610612763', ['2019-20','2020-21','2021-22'])
drose = Player('Derrick Rose', '201565', '1610612741', ['2008-09','2009-10','2010-11'])
players = [ja, drose]

Processing data for lollipop plots

In [4]:
# Visualizing data for the season averages
pd.read_json('../data/' + players[0].nbaid() + '/season_avg_per100.json')

,By Year,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,OffRtg,DefRtg,NetRtg,AST%,AST/TO,AST Ratio,OREB%,DREB%,REB%,TO Ratio,eFG%,TS%,USG%,PACE,PIE,PTS OFF TO,2nd PTS,FBPs,PITP,OppPTS OFF TO,Opp2nd PTS,OppFBPs,OppPITP,BLKA,PFD,%FGA2PT,%FGA3PT,%PTS2PT,%PTS2PT MR,%PTS3PT,%PTSFBPs,%PTSFT,%PTSOffTO,%PTSPITP,2FGM%AST,2FGM%UAST,3FGM%AST,3FGM%UAST,FGM%AST,FGM%UAST,%FGM,%FGA,%3PM,%3PA,%FTM,%FTA,%OREB,%DREB,%REB,%AST,%TOV,%STL,%BLK,%BLKA,%PF,%PFD,%PTS
14,2010-11,CHI,81,52.1,34.9,12.2,27.5,44.5,2.2,6.6,33.2,8.2,9.6,85.8,1.4,4.3,5.7,10.7,4.8,1.5,0.9,2.3,60.0,23,1,8.6,109.1,100.7,8.5,37.2,2.24,22.8,2.8,8.2,5.6,10.2,48.5,55.0,31.4,92.07,16.5,5.5,2.9,5.9,13.9,17.1,13.3,14.4,40.6,2.2,6.7,75.9,24.1,57.6,17.8,19.0,17.0,23.4,15.8,39.8,20.4,79.6,59.4,40.6,27.4,72.6,29.8,31.3,31.8,34.2,41.0,36.3,10.9,12.3,11.9,44.4,32.3,20.2,15.3,35.2,11.3,32.1,32.0
15,2009-10,CHI,78,50.9,28.7,11.9,24.3,48.9,0.3,1.1,26.7,4.6,6.0,76.6,1.2,4.0,5.2,8.3,3.8,1.0,0.5,1.7,48.0,6,0,-0.7,103.2,103.6,-0.4,29.7,2.16,21.2,2.3,7.6,5.0,9.8,49.5,53.2,26.8,94.48,13.7,4.0,2.1,5.2,13.0,15.3,13.6,14.0,43.6,1.6,4.6,95.6,4.4,81.0,35.6,3.0,18.0,16.0,14.0,45.5,30.9,69.1,56.3,43.8,31.5,68.5,29.9,27.7,6.8,8.9,23.8,23.4,9.3,11.5,10.9,38.6,26.3,15.6,7.6,28.1,8.5,21.8,27.8
16,2008-09,CHI,81,50.8,23.0,9.7,20.5,47.5,0.3,1.2,22.2,3.3,4.2,78.8,1.6,3.7,5.4,8.7,3.4,1.1,0.3,2.1,43.3,8,0,-2.0,106.9,108.6,-1.8,28.3,2.53,25.2,3.3,7.6,5.4,10.0,48.2,51.6,22.2,94.64,11.3,3.8,2.1,3.9,11.0,18.1,16.7,14.0,47.7,1.5,3.8,94.0,6.0,82.0,34.1,3.5,17.1,14.5,16.5,47.9,34.2,65.8,81.3,18.8,35.5,64.5,24.1,23.3,4.6,8.0,16.4,16.7,13.0,11.9,12.2,39.1,22.3,14.5,5.1,25.8,9.9,18.2,21.6


In [19]:
# Merging data of interest into a single feather dataframe for visualizations in R
data = pd.DataFrame()
for player in players:
    player_data = pd.read_json('../data/' + player.nbaid() + '/season_avg_per100.json')
    player_data['PLAYER'] = [player.name()]*len(player_data)
    player_data['SEASON'] = range(3,0,-1)
    data = pd.concat([data, player_data[['PLAYER', 'SEASON', 'PTS', 'AST', 'REB', 'TOV', 'STL', 'BLK', 'OffRtg', 'DefRtg', 'TS%', 'USG%', '%PTS2PT', '%PTSPITP', '%PTSFBPs', 'FGM%UAST']]],
                    axis=0)
                    
cols = data.columns
data.columns = cols.str.replace('%', 'pct')
data = data.sort_values(by='SEASON')
feather.write_dataframe(data,'./Rdata/stats.feather')

Processing data for scatterplots

In [21]:
# Changing column names and writing into feather 
df = pd.read_json('../data/byseason/2021-22guards.json')
cols = df.columns
df.columns = cols.str.replace('%', 'pct')
df['FGM2pctUAST'] = df['2FGMpctUAST']
df.drop('2FGMpctUAST', axis = 1, inplace = True)

feather.write_dataframe(df, './Rdata/2021-22.feather')

In [25]:
# EDA
df.sort_values(by = ['PTS'], ascending = False)[['PLAYER', 'PTS', 'pctPTS2PT']]

,PLAYER,PTS,pctPTS2PT
0,Luka Doncic,40.1,47.6
1,Ja Morant,39.1,62.9
2,Trae Young,38.8,44.4
3,Donovan Mitchell,37.2,44.4
4,DeMar DeRozan,37.0,68.4
...,...,...,...
209,Rodney Hood,11.0,34.4
210,Tony Snell,10.9,25.7
211,Tomas Satoransky,10.7,61.6
212,Andre Iguodala,9.9,46.8


In [ ]:
# Changing column names and writing into feather 
df = pd.read_json('../data/byseason/2010-11guards.json')
cols = df.columns
df.columns = cols.str.replace('%', 'pct')
df['FGM2pctUAST'] = df['2FGMpctUAST']
df.drop('2FGMpctUAST', axis = 1, inplace = True)

feather.write_dataframe(df, './Rdata/2010-11.feather')

Processing data for shot_plots

In [4]:
# Merging all shots into a single dataframe
for player in players:
    allshots = pd.DataFrame()
    for season in player.nbaseasons():
        allshots = pd.concat([allshots, pd.read_json('../data/' + player.nbaid() + '/' + season + '_shots.json')], axis=0)
    feather.write_dataframe(allshots, './Rdata/'+ player.nbaid() +'_allshots.feather')